# Required libraries

In [ ]:
!nvidia-smi

In [ ]:
# Takes about 2 minutes
!pip install -q fastapi uvicorn pyngrok pydantic bitsandbytes>=0.39.0

In [ ]:
# Compile and install with CPU support (Takes about 3 minutes)
!pip install llama-cpp-python

In [ ]:
# Compile and install with GPU support (Takes about 30 minutes)
!CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install -v llama-cpp-python

In [ ]:
# Prebuilt for CPU (Takes less than a minute) (Doesn't work on colab)
!pip install llama-cpp-python==0.3.2 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu
# Don't change version !!! (Not all versions have prebuilt wheels, check link above for latest version)

In [ ]:
# Prebuilt for GPU (Takes less than a minute)
!pip install -q llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
# Don't change version !!! (Not all versions have prebuilt wheels, check link above for latest version)

# Download the model

In [ ]:
# from llama_cpp import Llama
# llm = Llama.from_pretrained(
#     repo_id="lmstudio-community/Qwen2.5-7B-Instruct-1M-GGUF",
#     filename="*Q4_K_M.gguf",
#     verbose=False
# )

# with aria2
# !apt-get -y install -qq aria2
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/mradermacher/Xwen-7B-Chat-GGUF/resolve/main/Xwen-7B-Chat.Q5_K_M.gguf?download=true -d /content/text-generation-webui/models/ -o Xwen-7B-Chat.Q5_K_M.gguf

!mkdir -p models
!wget -O models/qwen2.5-7b-q4_k_m.gguf https://huggingface.co/lmstudio-community/Qwen2.5-7B-Instruct-1M-GGUF/resolve/main/Qwen2.5-7B-Instruct-1M-Q4_K_M.gguf

# Prepare the app.py file that has the LLM initialization and API methods

## Imports and loading the model

In [ ]:
%%writefile app.py
import random
from pyngrok import ngrok
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from llama_cpp import Llama
import uvicorn
import json
import requests

# Load the model
model_path = "models/qwen2.5-7b-q4_k_m.gguf"
llm = Llama(model_path=model_path, n_ctx=2048, n_threads=4, n_gpu_layers=-1)

## Conversation class

In [ ]:
%%writefile -a app.py
class Conversation:
    def __init__(self, llm: Llama, system_prompt="", history=[]):
        self.llm = llm
        self.system_prompt = system_prompt
        self.history = [{"role": "system", "content": self.system_prompt}] + history
    def create_completion(self, user_prompt=''):
        # Add the user prompt to the history
        self.history.append({"role": "user", "content": user_prompt})
        # Send the history messages to the LLM
        output = self.llm.create_chat_completion(messages=self.history, temperature=0.4, max_tokens=500)
        conversation_result = output['choices'][0]['message']
        # Append the conversation_result to the history
        self.history.append(conversation_result)
        return conversation_result['content']

    def final_response(self, response_format):
        output = self.llm.create_chat_completion(messages=self.history, response_format=response_format)
        try:
            return json.loads(output["choices"][0]["message"]["content"].strip())  # Parse JSON response
        except json.JSONDecodeError:
            return [{"condition": "Error parsing response", "confidence": "N/A","output":output}]

    def delete_history(self):
        self.history=[{"role": "system", "content": self.system_prompt}]

    def get_history(self):
        return self.history

## Initializing Conversation

In [ ]:
%%writefile -a app.py
system_prompt="You are a helpful dentist assistant that asks precise questions to diagnose dental conditions based on conversations."
chatbot = Conversation(llm, system_prompt=system_prompt)

## FastAPI initialization and API methods

In [ ]:
%%writefile -a app.py
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173", "*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class SymptomRequest(BaseModel):
    symptoms: list[str]
    additional_details: str = "None"

class DiagnosisResponse(BaseModel):
    diagnosis: list[dict]

class ChatHistoryRequest(BaseModel):
    chat_history: list[dict]

class SymptomsResponse(BaseModel):
    symptoms: list[dict]

class ConditionsResponse(BaseModel):
    diagnosis: list[dict]

class SummaryResponse(BaseModel):
    summary: str

class CombinedResponse(BaseModel):
    symptoms: list[dict]
    conditions: list[dict]
    summary: str

@app.get("/hello")
async def root():
    return {"message": "Hello World"}

@app.post("/diagnose-en", response_model=DiagnosisResponse)
async def diagnose_patient_lm(request: SymptomRequest):
    try:
        messages = prepare_prompt_en(request.symptoms, request.additional_details)
        response_format = get_conditions_json_response_format()
        response = query_local_model(messages, response_format)
        return {"diagnosis": response["diagnosis"]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/diagnose-fr", response_model=DiagnosisResponse)
async def diagnose_patient_fr(request: SymptomRequest):
    try:
        messages = prepare_prompt_fr(request.symptoms, request.additional_details)
        response_format = get_conditions_json_response_format()
        response = query_local_model(messages, response_format)
        return {"diagnosis": response["diagnosis"]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/converse")
async def converse(request):
    try:
        response=chatbot.create_completion(request)
        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/converse_diagnose")
async def converse_diagnose():
    try:
        response_format = get_conditions_json_response_format()
        response=chatbot.final_response(response_format)
        chatbot.delete_history()
        return {"diagnosis": response["diagnosis"]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/converse_history")
async def converse_history():
    try:
        response = chatbot.get_history()
        return {"history": response[1:]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/random_greeting")
async def get_random_greeting():
    greetings=["Hello! Welcome to our practice. How can I assist you today?",
           "Good morning/afternoon! Thank you for visiting us. What brings you in today?",
           "Hi there! I hope you're feeling well. How can I help you?",
           "Hello! We're glad you're here. What can I do for you today?",
           "Good day! I see you're here. What would you like to discuss?",
           "Hi! Thank you for coming in. How can I assist you?",
           "Hello! We're here to help. What can I do for you today?",
           "Hi! I hope you're feeling well. What can I do for you today?",
           "Good day! We're glad you're here. What brings you in today?"]
    try:
        return {"greeting": random.choice(greetings)}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

### Chat summary, symptoms and conditions FastAPI endpoints

In [ ]:
%%writefile -a app.py
@app.post("/extract_symptoms", response_model=SymptomsResponse)
async def extract_symptoms(request: ChatHistoryRequest):
    try:
        response = extract_symptoms_from_history(request.chat_history)
        return {"symptoms": response["symptoms"]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/extract_conditions", response_model=ConditionsResponse)
async def extract_conditions(request: ChatHistoryRequest):
    try:
        response = extract_conditions_from_history(request.chat_history)
        return {"diagnosis": response["diagnosis"]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/summarize", response_model=SummaryResponse)
async def summarize_chat(request: ChatHistoryRequest):
    try:
        SummaryResponse.summary = summarize_chat_history(request.chat_history)
        return SummaryResponse
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Updated /process_chat endpoint using helper functions
@app.post("/process_chat", response_model=CombinedResponse)
async def process_chat(request: ChatHistoryRequest):
    try:
        # Extract all required data using helper functions
        symptoms_response = extract_symptoms_from_history(request.chat_history)
        conditions_response = extract_conditions_from_history(request.chat_history)
        summary = summarize_chat_history(request.chat_history)

        # Combine results
        return CombinedResponse(
            symptoms=symptoms_response.get("symptoms", []),
            conditions=conditions_response.get("diagnosis", []),
            summary=summary
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/chat")
async def chat_with_model(request: ChatHistoryRequest):
    chat_system_prompt = "You are a helpful dentist assistant that asks precise questions to diagnose dental conditions based on conversations."
    messages = (
        [{"role": "system", "content": chat_system_prompt}]
        + request.chat_history
    )
    response = query_local_model(messages)
    return {"response": response}

### Improve doctor note

In [ ]:
%%writefile -a app.py
# Request model for the note improvement endpoint
class ImproveNoteRequest(BaseModel):
    etat: str
    doctor_note: str
    chat_history: list[dict]

# Response model for the improved note
class ImprovedNoteResponse(BaseModel):
    improved_note: str

# Helper function to improve the doctor's note
def improve_doctor_note(etat: str, doctor_note: str, chat_history: list[dict]) -> str:
    system_prompt = (
        "You are a professional medical assistant specializing in refining doctor’s notes. "
        "Your task is to improve the clarity, grammar, and readability of the provided doctor’s note "
        "while preserving its original meaning and intent. Do not add new information, remove content, "
        "or alter the medical conclusions. Fix typos, adjust awkward phrasing, and ensure a polished, "
        "professional tone. Use the consultation status and conversation for context, but do not incorporate "
        "details from them into the note unless they are already present in the doctor’s note."
    )
    user_prompt = (
        f"The consultation has the status: {etat}.\n"
        f"Here is the doctor’s note to improve:\n{doctor_note}\n"
        "Please provide the improved version of the note, keeping the meaning unchanged."
    )
    messages = (
        [{"role": "system", "content": system_prompt}]
        + chat_history
        + [{"role": "user", "content": user_prompt}]
    )
    return query_local_model(messages)  # Returns plain text

# New endpoint to improve the doctor's note
@app.post("/improve_note", response_model=ImprovedNoteResponse)
async def improve_note(request: ImproveNoteRequest):
    try:
        improved_note = improve_doctor_note(
            etat=request.etat,
            doctor_note=request.doctor_note,
            chat_history=request.chat_history
        )
        return {"improved_note": improved_note}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

## Prepare prompt function <br> returns: messages, response_format

In [ ]:
%%writefile -a app.py
def prepare_prompt_en(symptoms, details):
    prompt= f"""
    A patient reports these dental symptoms: {', '.join(symptoms)}.
    Additional details: {details}.

    Provide the 3 most probable dental conditions with confidence percentages.
    """

    messages=[
        {
            "role": "system",
            "content": "You are a helpful dentist assistant that diagnoses conditions based on symptoms and that outputs in JSON.",
        },
        {
            "role": "user",
            "content": prompt},
    ]
    return messages

def prepare_prompt_fr(symptoms, details):
    prompt= f"""
    A patient reports these dental symptoms: {', '.join(symptoms)}.
    Additional details: {details}.

    Provide the 3 most probable dental conditions in french with confidence percentages.
    """

    messages=[
        {
            "role": "system",
            "content": "You are a helpful dentist assistant that speaks french and diagnoses conditions based on symptoms and that outputs in JSON.",
        },
        {
            "role": "user",
            "content": prompt},
    ]
    return messages

### Add System prompt and append user prompt (Not used but could be useful to simplify functions)

In [ ]:
# %%writefile -a app.py
# def add_system_prompt_to_chat_history(system_prompt, chat_history_without_system_prompt):
#   messages=[]
#   system_prompt_element={
#       "role": "user",
#       "content": system_prompt
#   }

#   messages.append(system_prompt_element)
#   chat_history_with_system_prompt= messages + chat_history_without_system_prompt

#   return chat_history_with_system_prompt

# def append_user_prompt_to_chat_history(chat_history_with_system_prompt, prompt):
#   prompt_element={
#       "role": "user",
#       "content": prompt
#   }

#   chat_history_with_system_prompt.append(prompt_element)

#   return chat_history_with_system_prompt

### Helper functions for summary and symptoms and conditions (They do the actual work)

In [ ]:
%%writefile -a app.py
def extract_symptoms_from_history(chat_history: list[dict]) -> dict:
    """
    Extracts symptoms from the chat history using the LLM.

    Args:
        chat_history: List of messages with 'role' and 'content' keys.

    Returns:
        Dictionary containing the extracted symptoms (e.g., {"symptoms": [...]})
    """
    symptoms_system_prompt = "You are a helpful dentist assistant that extracts symptoms based on conversations and exports to JSON."
    symptoms_user_prompt = "Can you please extract patient symptoms from the conversation for the doctor to review"
    messages = (
        [{"role": "system", "content": symptoms_system_prompt}]
        + chat_history
        + [{"role": "user", "content": symptoms_user_prompt}]
    )
    response_format = get_symptoms_json_response_format()
    response = query_local_model(messages, response_format)
    return response

def extract_conditions_from_history(chat_history: list[dict]) -> dict:
    """
    Extracts conditions from the chat history using the LLM.

    Args:
        chat_history: List of messages with 'role' and 'content' keys.

    Returns:
        Dictionary containing the extracted conditions (e.g., {"diagnosis": [...]})
    """
    conditions_system_prompt = "You are a helpful dentist assistant that determines dental conditions based on conversations and exports to JSON."
    conditions_user_prompt = "Can you please determine what are the most probable dental conditions based on what I told you along with confidence percentages for each condition"
    messages = (
        [{"role": "system", "content": conditions_system_prompt}]
        + chat_history
        + [{"role": "user", "content": conditions_user_prompt}]
    )
    response_format = get_conditions_json_response_format()
    response = query_local_model(messages, response_format)
    return response

def summarize_chat_history(chat_history: list[dict]) -> str:
    """
    Summarizes the chat history into a concise text summary.

    Args:
        chat_history: List of messages with 'role' and 'content' keys.

    Returns:
        A string containing the summary.
    """
    summary_system_prompt = "You are a helpful dentist assistant that summarizes patient concerns, habits and medical history if possible based on conversations between a patient and an LLM."
    summary_user_prompt = "Can you please summarize the whole conversation for the doctor without including any symptoms or conditions. Don't say anything apart from the summary"
    messages = (
        [{"role": "system", "content": summary_system_prompt}]
        + chat_history
        + [{"role": "user", "content": summary_user_prompt}]
    )
    summary = query_local_model(messages)
    return summary

## Get json response format functions

In [ ]:
%%writefile -a app.py
def get_conditions_json_response_format():
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "diagnosis": {
                    "type": "array",
                    "items": {
                        "type":"object",
                        "properties": {
                            "condition": {"type": "string"},
                            "confidence": {"type": "integer"}
                            },
                        "required": ["condition","confidence"],
                        },
                    "minItems": 1,
                    "maxItems": 3
                    },
                },
            "required": ["diagnosis"]
            }
        }
    return response_format

def get_symptoms_json_response_format():
  response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "symptoms": {
                    "type": "array",
                    "items": {
                        "type":"object",
                        "properties": {
                            "symptom": {"type": "string"}
                            },
                        "required": ["symptom"],
                        },
                    "minItems": 1,
                    "maxItems": 10
                    },
                },
            "required": ["symptoms"]
            }
        }
  return response_format

## Upload ngrok url to gist function

In [ ]:
import requests
import json

def update_ngrok_url_in_gist(ngrok_url):
  GIST_ID = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Replace with your gist ID
  GITHUB_TOKEN = "github_pat_token_xxx"  # Replace with your token
  headers = {
      "Authorization": f"token {GITHUB_TOKEN}",
      "Accept": "application/vnd.github.v3+json"
  }
  data = {
      "files": {
          "ngrok_url.txt": {
              "content": ngrok_url
          }
      }
  }
  response = requests.patch(
      f"https://api.github.com/gists/{GIST_ID}",
      headers=headers,
      data=json.dumps(data)
  )
  if response.status_code == 200:
    return True
  else:
    return False


## Query local model method <br> Calls the llm.create_chat_completions method and returns the output

In [ ]:
%%writefile -a app.py
def query_local_model(messages, response_format=None):
  output = llm.create_chat_completion(
    messages=messages,
    response_format=response_format,
    temperature=0.3,
    max_tokens=256,
    )
  content = output["choices"][0]["message"]["content"].strip()

  # If response_format is provided, parse as JSON; otherwise, return as plain text
  if response_format:
    try:
      return json.loads(content)
    except json.JSONDecodeError:
      return {"error": "Failed to parse JSON response", "output": content}
  else:
    return content

if __name__ == "__main__":
    ngrok.set_auth_token("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")  # Replace with your token
    # public_url = ngrok.connect(8000).public_url
    # print(f" * Test all endpoints interactively at: {public_url}/docs")
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Lancer le serveur
## Cliquer sur l'url affiché (qui se termine par /docs) pour tester l'api

In [ ]:
!python app.py

In [ ]:
import subprocess
from pyngrok import ngrok

# Start the FastAPI server in the background
process = subprocess.Popen(["python", "app.py"])

# Expose the server using ngrok
ngrok.set_auth_token("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")  # Replace with your token
public_url = ngrok.connect(8000).public_url
update_response = update_ngrok_url_in_gist(public_url)
print(f" * Updated Gist with new ngrok URL: {update_response}")
print(f" * ngrok tunnel/API URL running at: {public_url}")
print(f" * Test all endpoints interactively at: {public_url}/docs")
# print(f"Nb!:Il faut remplacer NGROK_URL dans .env dans le projet FastAPI par cet url")

# Keep the cell alive to prevent the process from terminating
try:
    while True:
        pass
except KeyboardInterrupt:
    process.terminate()

# Testing summarize the patient consultation

In [ ]:
from llama_cpp import Llama

# Load the model
model_path = "models/qwen2.5-7b-q4_k_m.gguf"
llm = Llama(model_path=model_path, n_ctx=2048, n_threads=4, n_gpu_layers=-1)
prompt = "Can you please summarize the whole conversation for the doctor without including any symptoms or conditions. Don't say anything apart from the summary"
messages=[
    {
      "role": "system",
      "content": "You are a helpful dentist assistant that summarizes patient concerns, habits and medical history if possible based on conversations between a patient and an LLM."
    },
    {
      "role": "user",
      "content": "hi"
    },
    {
      "role": "assistant",
      "content": "Hello! I'm here to help with any dental concerns you might have. Could you describe what's bringing you in today? Are you experiencing any pain, discomfort, or noticing something unusual with your teeth or gums?"
    },
    {
      "role": "user",
      "content": "yes i'm feeling pain in my teeth especially when i eat cold things"
    },
    {
      "role": "assistant",
      "content": "Thank you for sharing that with me. The pain you're experiencing when eating cold things could be a sign of several dental issues. To help diagnose the problem, could you answer a few questions?\n\n1. How long have you been experiencing this pain?\n2. Do you have any other symptoms like sensitivity to heat, discoloration of the tooth, or swelling in the gums?\n3. Have you noticed any changes in your teeth recently, such as chips or cracks?\n4. Do you have any fillings or crowns in your teeth?\n5. How often do you brush your teeth, and do you use a soft-bristled toothbrush?\n\nThis information will help me better understand your situation."
    },
    {
      "role": "user",
      "content": "i use a soft toothbrush and brush regularly and the pain started about 2 weeks ago and it's only getting worse along with swelling in my gums"
    },
    {
      "role": "assistant",
      "content": "Thank you for providing those additional details. Based on what you've described, the combination of pain, especially with cold things, swelling in the gums, and the recent onset of symptoms could indicate a few potential issues:\n\n1. **Dental Decay (Cavities):** The pain when eating cold things could be due to decay that has reached the pulp of the tooth, causing inflammation and sensitivity.\n\n2. **Gum Infection (Gingivitis or Periodontitis):** The swelling in your gums could be due to an infection, which is worsening over time. This could be related to poor plaque removal or other underlying issues.\n\n3. **Tooth Fracture or Crack:** If you've had any trauma to your teeth, a crack might have developed, leading to sensitivity and pain, especially with cold stimuli.\n\n4. **Reversible Pulpitis:** This is inflammation of the tooth's pulp that can be reversible if caught early. However, if left untreated, it can lead to irreversible pulpitis or an abscess.\n\nGiven that the pain is worsening and now includes swelling, it's important to see a dentist as soon as possible. The swelling could indicate an infection that needs to be addressed promptly to prevent further complications.\n\nWould you be able to make an appointment with a dentist soon? They will be able to perform a thorough examination and possibly take X-rays to determine the exact cause of your symptoms. In the meantime, you can try to avoid very cold foods and drinks to minimize discomfort.\n\nIf you have any more questions or need further assistance, feel free to ask!"
    },
    {
      "role": "user",
      "content": prompt
    }
  ]

output = llm.create_chat_completion(
    messages=messages,
    temperature=0.4,
    max_tokens=128,
    )

In [ ]:
print(output["choices"][0]["message"]["content"])

# Testing extract symptoms from chat

In [ ]:
from llama_cpp import Llama

# Load the model
model_path = "models/qwen2.5-7b-q4_k_m.gguf"
llm = Llama(model_path=model_path, n_ctx=2048, n_threads=4, n_gpu_layers=-1)
prompt = "Can you please extract patient symptoms from the conversation for the doctor to review"
messages=[
    {
      "role": "system",
      "content": "You are a helpful dentist assistant that extracts symptoms based on conversations and exports to JSON."
    },
    {
      "role": "user",
      "content": "hi"
    },
    {
      "role": "assistant",
      "content": "Hello! I'm here to help with any dental concerns you might have. Could you describe what's bringing you in today? Are you experiencing any pain, discomfort, or noticing something unusual with your teeth or gums?"
    },
    {
      "role": "user",
      "content": "yes i'm feeling pain in my teeth especially when i eat cold things"
    },
    {
      "role": "assistant",
      "content": "Thank you for sharing that with me. The pain you're experiencing when eating cold things could be a sign of several dental issues. To help diagnose the problem, could you answer a few questions?\n\n1. How long have you been experiencing this pain?\n2. Do you have any other symptoms like sensitivity to heat, discoloration of the tooth, or swelling in the gums?\n3. Have you noticed any changes in your teeth recently, such as chips or cracks?\n4. Do you have any fillings or crowns in your teeth?\n5. How often do you brush your teeth, and do you use a soft-bristled toothbrush?\n\nThis information will help me better understand your situation."
    },
    {
      "role": "user",
      "content": "i use a soft toothbrush and brush regularly and the pain started about 2 weeks ago and it's only getting worse along with swelling in my gums"
    },
    {
      "role": "assistant",
      "content": "Thank you for providing those additional details. Based on what you've described, the combination of pain, especially with cold things, swelling in the gums, and the recent onset of symptoms could indicate a few potential issues:\n\n1. **Dental Decay (Cavities):** The pain when eating cold things could be due to decay that has reached the pulp of the tooth, causing inflammation and sensitivity.\n\n2. **Gum Infection (Gingivitis or Periodontitis):** The swelling in your gums could be due to an infection, which is worsening over time. This could be related to poor plaque removal or other underlying issues.\n\n3. **Tooth Fracture or Crack:** If you've had any trauma to your teeth, a crack might have developed, leading to sensitivity and pain, especially with cold stimuli.\n\n4. **Reversible Pulpitis:** This is inflammation of the tooth's pulp that can be reversible if caught early. However, if left untreated, it can lead to irreversible pulpitis or an abscess.\n\nGiven that the pain is worsening and now includes swelling, it's important to see a dentist as soon as possible. The swelling could indicate an infection that needs to be addressed promptly to prevent further complications.\n\nWould you be able to make an appointment with a dentist soon? They will be able to perform a thorough examination and possibly take X-rays to determine the exact cause of your symptoms. In the meantime, you can try to avoid very cold foods and drinks to minimize discomfort.\n\nIf you have any more questions or need further assistance, feel free to ask!"
    },
        {
      "role": "user",
      "content": prompt
    }
  ]

response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "symptoms": {
                    "type": "array",
                    "items": {
                        "type":"object",
                        "properties": {
                            "symptom": {"type": "string"}
                            },
                        "required": ["symptom"],
                        },
                    "minItems": 1,
                    "maxItems": 10
                    },
                },
            "required": ["symptoms"]
            }
        }

output = llm.create_chat_completion(
    messages=messages,
    response_format=response_format,
    temperature=0.4,
    max_tokens=256,
    )

In [ ]:
print(output["choices"][0]["message"]["content"])

# Testing extract conditions from chat

In [ ]:
from llama_cpp import Llama

# Load the model
model_path = "models/qwen2.5-7b-q4_k_m.gguf"
llm = Llama(model_path=model_path, n_ctx=2048, n_threads=4, n_gpu_layers=-1)
prompt = "Can you please extract the conditions the patient is feeling from the conversation for the doctor"
messages=[
    {
      "role": "system",
      "content": "You are a helpful dentist assistant that determines dental conditions based on conversations and exports to JSON."
    },
    {
      "role": "user",
      "content": "hi"
    },
    {
      "role": "assistant",
      "content": "Hello! I'm here to help with any dental concerns you might have. Could you describe what's bringing you in today? Are you experiencing any pain, discomfort, or noticing something unusual with your teeth or gums?"
    },
    {
      "role": "user",
      "content": "yes i'm feeling pain in my teeth especially when i eat cold things"
    },
    {
      "role": "assistant",
      "content": "Thank you for sharing that with me. The pain you're experiencing when eating cold things could be a sign of several dental issues. To help diagnose the problem, could you answer a few questions?\n\n1. How long have you been experiencing this pain?\n2. Do you have any other symptoms like sensitivity to heat, discoloration of the tooth, or swelling in the gums?\n3. Have you noticed any changes in your teeth recently, such as chips or cracks?\n4. Do you have any fillings or crowns in your teeth?\n5. How often do you brush your teeth, and do you use a soft-bristled toothbrush?\n\nThis information will help me better understand your situation."
    },
    {
      "role": "user",
      "content": "i use a soft toothbrush and brush regularly and the pain started about 2 weeks ago and it's only getting worse along with swelling in my gums"
    },
    {
      "role": "assistant",
      "content": "Thank you for providing those additional details. Based on what you've described, the combination of pain, especially with cold things, swelling in the gums, and the recent onset of symptoms could indicate a few potential issues:\n\n1. **Dental Decay (Cavities):** The pain when eating cold things could be due to decay that has reached the pulp of the tooth, causing inflammation and sensitivity.\n\n2. **Gum Infection (Gingivitis or Periodontitis):** The swelling in your gums could be due to an infection, which is worsening over time. This could be related to poor plaque removal or other underlying issues.\n\n3. **Tooth Fracture or Crack:** If you've had any trauma to your teeth, a crack might have developed, leading to sensitivity and pain, especially with cold stimuli.\n\n4. **Reversible Pulpitis:** This is inflammation of the tooth's pulp that can be reversible if caught early. However, if left untreated, it can lead to irreversible pulpitis or an abscess.\n\nGiven that the pain is worsening and now includes swelling, it's important to see a dentist as soon as possible. The swelling could indicate an infection that needs to be addressed promptly to prevent further complications.\n\nWould you be able to make an appointment with a dentist soon? They will be able to perform a thorough examination and possibly take X-rays to determine the exact cause of your symptoms. In the meantime, you can try to avoid very cold foods and drinks to minimize discomfort.\n\nIf you have any more questions or need further assistance, feel free to ask!"
    },
        {
      "role": "user",
      "content": prompt
    }
  ]

response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "diagnosis": {
                    "type": "array",
                    "items": {
                        "type":"object",
                        "properties": {
                            "condition": {"type": "string"},
                            "confidence": {"type": "integer"}
                            },
                        "required": ["condition","confidence"],
                        },
                    "minItems": 1,
                    "maxItems": 3
                    },
                },
            "required": ["diagnosis"]
            }
        }

output = llm.create_chat_completion(
    messages=messages,
    response_format=response_format,
    temperature=0.4,
    max_tokens=256,
    )

In [ ]:
print(output["choices"][0]["message"]["content"])

# Experimental

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct-1M"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [ ]:
print(response)